## DATA 602 - HW4
### Name: Precious Worgu
### Student ID: 119343890

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn import model_selection

In [2]:
file = '/Users/preciousworgu/Desktop/untitled folder/foods.txt'
df = pd.read_csv(file, sep='\t', encoding = 'ISO-8859-1')
df.loc[-1] = df.columns.values
df.sort_index(inplace=True)
df.reset_index(drop=True, inplace=True)
df = df['product/productId: B001E4KFG0'].str.split(':', expand=True)
df = df.T
df.columns = df.iloc[0]
df = df[1:]
df = df.iloc[:1]

df

,product/productId,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text,product/productId,review/userId,...,review/summary,review/text,product/productId,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text
1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1/1,5.0,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned ...,B00813GRG4,A1D87F6ZCVE5NK,...,Favorite Training and reward treat,These are the BEST treats for training and re...,B001LR2CU2,A3LGQPJCZVL9UC,srfell17,0/0,5.0,1338422400,Great Honey,"I am very satisfied ,product is as advertised..."


In [3]:
ProductID = df['product/productId'].melt()
ProductID = ProductID.drop(ProductID.columns[[0]], axis = 1)
ProductID = ProductID.rename(columns={'value':'ProductID'})
UserID = df['review/userId'].melt()
UserID = UserID.drop(UserID.columns[[0]], axis = 1)
UserID = UserID.rename(columns={'value':'UserID'})
ProfileName = df['review/profileName'].melt()
ProfileName = ProfileName.drop(ProfileName.columns[[0]], axis = 1)
ProfileName = ProfileName.rename(columns={'value':'ProfileName'})
Helpfulness = df['review/helpfulness'].melt()
Helpfulness = Helpfulness.drop(Helpfulness.columns[[0]], axis = 1)
Helpfulness = Helpfulness.rename(columns={'value':'Helpfulness'})
Score = df['review/score'].melt()
Score = Score.drop(Score.columns[[0]], axis = 1)
Score = Score.rename(columns={'value':'Score'})
Time = df['review/time'].melt()
Time = Time.drop(Time.columns[[0]], axis = 1)
Time = Time.rename(columns={'value':'Time'})
Summary = df['review/summary'].melt()
Summary = Summary.drop(Summary.columns[[0]], axis = 1)
Summary = Summary.rename(columns={'value':'Summary'})
Text = df['review/text'].melt()
Text = Text.drop(Text.columns[[0]], axis = 1)
Text = Text.rename(columns={'value':'Text'})



new_df = pd.concat([ProductID, UserID, ProfileName, Helpfulness, Score, Time, Summary, Text], axis="columns")
new_df["ID"] = new_df.index + 1
new_cols = ["ID", "ProductID", "UserID", "ProfileName", "Helpfulness", "Score", "Time", "Summary", "Text"]
new_df=new_df[new_cols]
new_df['Score'] = new_df['Score'].astype(float)
new_df["Score"] = np.where(new_df["Score"] >= 4, "positive", "negative")
new_df

,ID,ProductID,UserID,ProfileName,Helpfulness,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1/1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned ...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0/0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanu...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1/1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a f...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3/3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient ...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0/0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wi...
...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0/0,positive,1299628800,Will not do without,Great for sesame chicken..this is a good if n...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0/0,negative,1331251200,disappointed,I'm disappointed with the flavor. The chocola...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2/2,positive,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 ..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1/1,positive,1331596800,Favorite Training and reward treat,These are the BEST treats for training and re...


In [4]:
stop = set(stopwords.words('english'))
snow = nltk.stem.SnowballStemmer('english')

def cleanhtml(sentence):
    cleaner = re.compile('<.*>')
    cleantext = re.sub(cleaner, ' ', sentence)
    return cleantext

def cleanpunc(sentence):
    cleaned = re.sub(r'[?|.|!|*|@|#|\'|"|,|)|(|\|/]', r'', sentence)
    return cleaned

i = 0
str1 = ''
all_positive_words = []
all_negative_words = []
final_string = []

s=''
for sent in new_df['Text'].values:
    filtered_sentence = []
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if ((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                if (cleaned_words.lower() not in stop):
                    s = (snow.stem(cleaned_words.lower())).encode('utf-8')
                    filtered_sentence.append(s)
                    if (new_df['Score'].values)[i] == 'positive':
                        all_positive_words.append(s)
                    if (new_df['Score'].values)[i] == 'negative':
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue
    str1 = b" ".join(filtered_sentence)
    final_string.append(str1)
    i += 1
    
new_df['CleanedText'] = final_string
new_df['CleanedText'] = new_df['CleanedText'].str.decode('utf-8')
new_df

,ID,ProductID,UserID,ProfileName,Helpfulness,Score,Time,Summary,Text,CleanedText
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1/1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned ...,bought sever vital can dog food product found ...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0/0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanu...,product arriv label jumbo salt peanutsth peanu...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1/1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a f...,confect around centuri light pillowi citrus ge...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3/3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient ...,look secret ingredi robitussin believ found go...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0/0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wi...,great taffi great price wide assort yummi taff...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0/0,positive,1299628800,Will not do without,Great for sesame chicken..this is a good if n...,great sesam chickenthi good better restur eate...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0/0,negative,1331251200,disappointed,I'm disappointed with the flavor. The chocola...,disappoint flavor chocol note especi weak milk...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2/2,positive,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 ...",star small give one train session tri train do...
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1/1,positive,1331596800,Favorite Training and reward treat,These are the BEST treats for training and re...,best treat train reward dog good groom lower c...


In [5]:
def naive_bayes(X_train , y_train):
    
    alpha_value = np.arange(1,500,0.5)
    cv_scores = []
    for alpha in alpha_value:
        mnb = MultinomialNB(alpha = alpha)
        scores = cross_val_score(mnb , X_train , y_train , cv = 10 , scoring = 'accuracy')
        cv_scores.append(scores.mean())
    MSE = [1 - x for x in cv_scores]
    optimal_alpha = alpha_value[MSE.index(min(MSE))]
    return optimal_alpha

vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(new_df['CleanedText'])
y = new_df["Score"]


X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.3)
classifier = MultinomialNB()
model=classifier.fit(X_train,y_train)
model.score(X_test,y_test)

0.8616312002673907